## Let's try move cancellation to simplify the existing solutions

- For all puzzles, following a move x by a move -x is equivalent to doing nothing, so remove these pairs

In [1]:
import ast
from collections import deque
import os
import re

import numpy as np
import pandas as pd
import tqdm

# path = '/kaggle/input/santa-2023/'
root = 'competition_data'
puzzles_df = pd.read_csv(os.path.join(root, 'puzzles.csv'))
puzzle_info_df = pd.read_csv(os.path.join(root, 'puzzle_info.csv'))
base_sub_df = pd.read_csv(os.path.join(root, 'sample_submission.csv'))

In [2]:
super_df = pd.merge(puzzles_df, puzzle_info_df, on='puzzle_type')
super_df = pd.merge(super_df, base_sub_df, on='id')

In [3]:
super_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              398 non-null    int64 
 1   puzzle_type     398 non-null    object
 2   solution_state  398 non-null    object
 3   initial_state   398 non-null    object
 4   num_wildcards   398 non-null    int64 
 5   allowed_moves   398 non-null    object
 6   moves           398 non-null    object
dtypes: int64(2), object(5)
memory usage: 21.9+ KB


In [4]:
# A utility class to help with some puzzle operations
class PermutationPuzzle(object):
    def __init__(self, puzzle_series: pd.Series):
        self.puzzle_type = puzzle_series["puzzle_type"]
        self.initial_state = np.array(puzzle_series["initial_state"].split(";"))
        self.solution_state = np.array(puzzle_series["solution_state"].split(";"))
        self.state = self.initial_state.copy()
        self.path = []
        self.num_wildcards = puzzle_series["num_wildcards"]
        self.base_solution = np.array(puzzle_series["moves"].split("."))

        # More fine-grained information about the puzzle type
        # puzzle_shape is either "cube", "globe", or "wreath"
        # dimensions is the puzzle size as a list of integers
        p_type_items = self.puzzle_type.split("_")
        self.puzzle_shape = p_type_items[0]
        numbers = p_type_items[1].split("/")
        self.dimensions = [int(k) for k in numbers]

        # Create the actions dictionaries
        # The expanded actions dictionary includes negative actions and repetitions of the same action
        # such that the best set of moves in every direction is included
        self.base_actions_dict = ast.literal_eval(puzzle_info_df.loc[puzzle_info_df['puzzle_type'] == self.puzzle_type, 'allowed_moves'].values[-1])
        self.actions_dict = {}
        # Need this object to use integers to index the actions, rather than the identifying string
        # Some off-the-shelf codes require actions to be integers, not strings
        # See also the self.actions_index_reverse object below...
        self.actions_index = {}

        if self.puzzle_shape == "wreath":
            # Figure out the rotations for wreath puzzles
            # For example, the 6x6 wreath has 5 possible left ring rotations: -2l, -l, l, 2l, and 3l (same for the right ring)
            # The 7x7 wreath has 6 possible left ring rotations: -3l, -2l, -l, l, 2l, and 3l (same for the right ring)
            # Use the min_rot and max_rot variables to determine the range of rotations and the
            # perm_nums list to enumerate each rotation
            n_moves_per_rot = self.dimensions[0] - 1
            max_rot = self.dimensions[0] // 2
            min_rot = -1 * max_rot
            is_odd_dim = self.dimensions[0] % 2 == 1
            if not is_odd_dim:
                min_rot += 1
            perm_nums = list(range(min_rot, max_rot + 1))
            perm_nums.remove(0)

            for i, (k, v) in enumerate(self.base_actions_dict.items()):
                perm = np.array(v)
                inv_perm = np.argsort(perm)

                for q, j in enumerate(perm_nums):
                    if j == 1:
                        new_k = k
                        new_v = perm.copy()
                    elif j == -1:
                        new_k = "-" + k
                        new_v = inv_perm.copy()
                    else:
                        new_k = str(j) + k
                        if j > 0:
                            new_v = perm.copy()
                            for _ in range(2, j+1):
                                new_v = new_v[perm]
                        else:
                            new_v = inv_perm.copy()
                            for _ in range(2, -j + 1):
                                new_v = new_v[inv_perm]

                    self.actions_dict[new_k] = new_v
                    self.actions_index[n_moves_per_rot*i + q] = new_k
        elif self.puzzle_shape == "cube":
            for i, (k, v) in enumerate(self.base_actions_dict.items()):
                # Add a second rotation in the same direction to the dictionary
                # The third rotation in the same direction would be the same as the negative of the first rotation, so do not add that one
                perm = np.array(v)
                k2 = '2' + k
                perm2 = perm[perm]

                self.actions_dict[k] = perm
                self.actions_dict[k2] = perm2
                self.actions_dict["-" + k] = np.argsort(perm)

                self.actions_index[3*i] = k
                self.actions_index[3*i+1] = k2
                self.actions_index[3*i+2] = "-" + k
        elif self.puzzle_shape == "globe":
            # The rotations of the lateral layers of the globe puzzle are similar to the ring rotations of the wreath puzzle
            # For example, each layer of the 3x4 globe has 7 possible rotations: -3r0, -2r0, -r0, r0, 2r0, 3r0, and 4r0
            # Notice that we have 4 rotations in one direction and (4-1) in the other direction
            n_layers = self.dimensions[0] + 1
            n_moves_per_rot = 2*self.dimensions[1] - 1
            n_layerwise_rotations = n_layers * n_moves_per_rot
            max_rot = self.dimensions[1]
            min_rot = -1 * max_rot + 1
            perm_nums = list(range(min_rot, max_rot + 1))
            perm_nums.remove(0)

            for i, (k, v) in enumerate(self.base_actions_dict.items()):
                perm = np.array(v)
                if "r" in k:
                    inv_perm = np.argsort(perm)

                    for q, j in enumerate(perm_nums):
                        if j == 1:
                            new_k = k
                            new_v = perm.copy()
                        elif j == -1:
                            new_k = "-" + k
                            new_v = inv_perm.copy()
                        else:
                            new_k = str(j) + k
                            if j > 0:
                                new_v = perm.copy()
                                for _ in range(2, j+1):
                                    new_v = new_v[perm]
                            else:
                                new_v = inv_perm.copy()
                                for _ in range(2, -j + 1):
                                    new_v = new_v[inv_perm]

                        self.actions_dict[new_k] = new_v
                        self.actions_index[n_moves_per_rot*i + q] = new_k
                elif "f" in k:
                    # The negative of the flip move is the same as the flip move, so do not add the negative move to the dictionary
                    # Further, in the puzzle description, the rotation moves are described before the flip moves. Therefore, since
                    # order is preserved in the base dictionary, the following indexing will work.
                    self.actions_dict[k] = perm
                    self.actions_index[n_layerwise_rotations + i - n_layers] = k
                else:
                    raise ValueError("Invalid action: {}".format(k))
        else:
            raise ValueError("Invalid puzzle shape: {}".format(self.puzzle_shape))
        
        # Utilities: a dictionary of indices and the length of the action set
        self.actions_index_reverse = {v: k for k, v in self.actions_index.items()}
        self.actions_dict_length = len(self.actions_dict)

    def render(self) -> None:
        # Print the current state in the same format as the initial and solution states
        print(";".join(self.state))
        return
    
    def get_state(self) -> list[str]:
        return self.state

    def reset_state(self) -> None:
        self.state = self.initial_state.copy()
        self.path = []
        return

    def possible_actions(self) -> list[int]:
        actions_list = list(np.arange(self.actions_dict_length))
        if len(self.path) == 0:
            return actions_list
        
        # Prevent the reverse of the action we just took from being selected right away
        last_move = self.path[-1]
        if self.puzzle_shape == "wreath":
            # For wreath puzzles, it doesn't make sense to follow a left ring rotation with another left ring rotation
            # (or follow a right ring rotation with another right ring rotation) because the second move will always be
            # suboptimal. Therefore, remove any rotations of the same ring from consideration
            ring_rotated = "l" if "l" in last_move else "r"
            remove_list = [k for k in self.actions_dict.keys() if ring_rotated in k]
        elif self.puzzle_shape == "cube":
            # For cube puzzles, it is also suboptimal to follow, e.g, a "d0" move with another "d0" move.
            # Furthermore, all "d" moves occur in the same plane, so we want to avoid the case where, e.g., a "d0" move
            # is followed by a "d1" is followed by another "d0" move. In addition, the "d0" and "d1" moves can be taken
            # in either order to reach the same state. To force an algorithm to choose one order over the other, we
            # force subsequent "d" moves to have a higher following integer, e.g., "d0" -> "d1" -> "d2" -> "d3".
            # (The reverse order would also be an acceptable option if desired.)
            # Therefore, e.g., if the previous move was in the "d1" plane, do not allow the next move to be in the "d0" or "d1" planes.
            # The same logic applies to the other planes, i.e., "f" and "l" moves.
            plane_rotated = last_move[-2]
            plane_rotated_num = int(last_move[-1])
            remove_list = [k for k in self.actions_dict.keys() if (plane_rotated in k and int(k[-1]) <= plane_rotated_num)]
        elif self.puzzle_shape == "globe":
            # For globe puzzles, the lateral layers follow the same logic as the planes in the cube puzzles.
            # E.g., if the previous move was in the "r1" plane, do not allow the next move to be in the "r0" or "r1" planes.
            # However, order matters for the flip moves. E.g., "f0" followed by "f1" is not the same as "f1" followed by "f0".
            # Therefore, if the last move is a flip, the remove list contains only that move.
            layer_rotated = last_move[-2]
            layer_rotated_num = int(last_move[-1])
            if layer_rotated == "r":
                remove_list = [k for k in self.actions_dict.keys() if ("r" in k and int(k[-1]) <= layer_rotated_num)]
            elif layer_rotated == "f":
                remove_list = [last_move]
            else:
                raise ValueError("Invalid action found: {}".format(last_move))
        else:
            raise ValueError("Invalid puzzle shape found: {}".format(self.puzzle_shape))

        for move in remove_list:
            actions_list.remove(self.actions_index_reverse[move])

        return actions_list
    
    def possible_actions_by_name(self) -> list[str]:
        if len(self.path) == 0:
            return list(self.actions_dict.keys())
        
        actions_list = self.possible_actions()
        return [self.actions_index[k] for k in actions_list]
    
    def take_action(self, action: int) -> None:
        # Add the action to the path
        self.path.append(self.actions_index[action])
        # Execute the permutation action
        perm = self.actions_dict[self.actions_index[action]]
        self.state = self.state[perm]
        return 
    
    def take_action_by_name(self, action: str) -> None:
        # Add the action to the path
        self.path.append(action)
        # Execute the permutation action
        perm = self.actions_dict[action]
        self.state = self.state[perm]
        return

    def is_terminated(self) -> bool:
        wc_count = 0
        for i in range(len(self.state)):
            if self.state[i] != self.solution_state[i]:
                wc_count += 1

        if wc_count > self.num_wildcards:
            return False
        else:
            return True

In [5]:
def remove_cancelling_pairs(base_solution):
    # Remove pairs of moves that cancel each other out
    list_flag = True    # Set to False when we have found no more pairs to remove
    move_flag = False   # Set to True when we want to skip a move because we just found a pair
    new_solution = deque(base_solution)
    while list_flag:
        list_flag = False
        old_solution = new_solution.copy()
        new_solution = deque()
        for i in range(1, len(old_solution)):
            if move_flag:
                move_flag = False
                continue
            elif (old_solution[i-1] == '-' + old_solution[i]) or ("-" + old_solution[i-1] == old_solution[i]):
                list_flag = True
                move_flag = True
            else:
                new_solution.append(old_solution[i-1])
        
        if not move_flag:
            new_solution.append(old_solution[-1])

    return list(new_solution)

In [6]:
# # Unit test
# test_move_list = ['l', '-l', 'r', 'r', 'r', '-r', 'r']
# test_move_list_better = remove_cancelling_pairs(test_move_list)
# print(test_move_list_better)

In [7]:
# Class unit tests
test_cube = PermutationPuzzle(super_df.loc[super_df['puzzle_type'] == 'cube_3/3/3', :].iloc[0])
test_globe = PermutationPuzzle(super_df.loc[super_df['puzzle_type'] == 'globe_1/8', :].iloc[0])
test_wreath = PermutationPuzzle(super_df.loc[super_df['puzzle_type'] == 'wreath_7/7', :].iloc[0])

In [8]:
# # Possible actions unit test
# a1 = test_cube.actions_index
# print(a1)
# test_cube.take_action(4)
# test_cube.possible_actions()
# test_cube.possible_actions_by_name()
# test_cube.take_action(10)
# test_cube.possible_actions()
# test_cube.possible_actions_by_name()
# test_cube.take_action(15)
# test_cube.possible_actions()
# test_cube.possible_actions_by_name()


In [9]:
for i, row in tqdm.tqdm(super_df.iterrows(), total=len(super_df)):
    puzzle_obj = PermutationPuzzle(row)
    base_sol_len = len(puzzle_obj.base_solution)

    new_solution = remove_cancelling_pairs(puzzle_obj.base_solution)
    new_sol_len = len(new_solution)

    if base_sol_len != new_sol_len:
        print(f"Base solution length: {base_sol_len}")
        print(f"New solution length: {new_sol_len}")

    # # For testing only
    # if i == 10:
    #     break

100%|██████████| 398/398 [00:15<00:00, 26.32it/s] 


In [11]:
# Splitting a string based on multiple characters, for example
s = "-2d0"
split_s = re.split("-|d|f|l|r", s)
print(split_s)

['', '2', '0']
